In [2]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import random

# lecture de la liste des profils bidons
profils = pd.read_csv('./Data/profiles2021.csv',sep=';', dtype=str)

###########################################
# boucle principale
###########################################
for i in range(0,2000):
    print(f'Itération {i}')

    profil = profils.loc[i]
        
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference('permissions.default.image', 2)

    # démarrage du browser avec une fenêtre très grande pour
    # qu'aucun champ du formulaire ne soit masqué
    browser=webdriver.Firefox(firefox_profile=firefox_profile)

    
    # processus d'inscription
    print("Vote pour ",profil['firsname']," ",profil['lastname'])  
    browser.get("https://budgetparticipatif.landes.fr/register")
    time.sleep(2)
    browser.set_window_size(1024, 4096)
    try:
        browser.find_element_by_id("tarteaucitronPersonalize").click()
    except:
        pass

    text_area = browser.find_element_by_id("app_register_type_email")
    text_area.send_keys(profil['email'])

    text_area = browser.find_element_by_id("app_register_type_username")
    text_area.send_keys(profil['username'])

    text_area = browser.find_element_by_id("app_register_type_plainPassword_first")
    text_area.send_keys(profil['plainPassword'])

    text_area = browser.find_element_by_id("app_register_type_plainPassword_second")
    text_area.send_keys(profil['plainPassword'])

    text_area = browser.find_element_by_id("app_register_type_profile_firstname")
    text_area.send_keys(profil['firsname'])

    text_area = browser.find_element_by_id("app_register_type_profile_lastname")
    text_area.send_keys(profil['lastname'])

    text_area = browser.find_element_by_id("app_register_type_profile_phone")
    text_area.send_keys(profil['phone'])

    text_area = browser.find_element_by_id("app_register_type_profile_address")
    text_area.send_keys(profil['address'])

    select = Select(browser.find_element_by_id('app_register_type_profile_gender'))
    select.select_by_value(profil['gender'])

    select = Select(browser.find_element_by_id('app_register_type_profile_date_of_birth_day'))
    select.select_by_value(profil['day'])
    select = Select(browser.find_element_by_id('app_register_type_profile_date_of_birth_month'))
    select.select_by_value(profil['month'])
    select = Select(browser.find_element_by_id('app_register_type_profile_date_of_birth_year'))
    select.select_by_value(profil['year'])

    browser.find_element_by_id("app_register_type_cguOptin").click()
    browser.find_element_by_id("app_register_type_profile_customOptin").click()

    # renseignement de la ville et du code postal (nouvelle procédure)
    browser.find_element_by_xpath('//span[@id="select2-app_register_type_profile_city-container"]').click()
    text_area = browser.find_element_by_xpath('//input[@class="select2-search__field"]')
    text_area.send_keys(profil.ville[:3])
    time.sleep(2)
    ville = browser.find_element_by_xpath('//ul[@id="select2-app_register_type_profile_city-results"]/li[position()=1]')
    postcode = ville.text[-6:-1]
    ville.click()
    text_area = browser.find_element_by_id("app_register_type_profile_postcode")
    text_area.send_keys(postcode)
    browser.find_element_by_id("app_register_type_save").click()
    browser.set_window_size(1024, 1024)
    
    # Validation mail, on laisse un peu de temps au mail d'arriver
    time.sleep(60)
    browser.get('https://yopmail.com?login='+profil['username'])
    time.sleep(1)
    try:
        browser.find_element_by_id("accept").click()
    except:
        pass
    
    browser.switch_to_frame("ifmail")
    url_valisation = "https://"+browser.find_element_by_id("mail").text.split("https://")[1].split('\n')[0]
    browser.get(url_valisation)
    browser.find_element_by_class_name('button-validate').click()
    
    # on se relogue
    browser.get("https://budgetparticipatif.landes.fr/login")
    text_area = browser.find_element_by_id("username")
    text_area.clear()
    text_area.send_keys(profil.email)

    text_area = browser.find_element_by_id("password")
    text_area.clear()
    text_area.send_keys(profil.plainPassword)

    # Envoi du formulaire
    browser.find_element_by_id("_submit").click()
    
    # vote
    projets = [line.rstrip('\n') for line in open('./Data/liste_projets.txt')]
    projetsNon_Concurrents = random.choices(projets,k=2)
    urls=[]
    urls.append("https://budgetparticipatif.landes.fr/dialog/budget-participatif-citoyen-2/proposal/ndeg16533-ouvrir-un-espace-fablab-pour-tous-a-saint-pierre-du-mont")
    urls.append(projetsNon_Concurrents[0])
    urls.append(projetsNon_Concurrents[1])
    
    # validation du vote pour chaque projet dont le notre
    for url in urls:
        browser.get(url)
        # Envoi du vote
        browser.find_element_by_id("app_vote_type_select").click()
        time.sleep(2)    
    browser.get("https://budgetparticipatif.landes.fr/usage/1595/action/1171/validate")
    
    # attente aléatoire, entre 1 minute et 10 minutes
    browser.quit()
    time.sleep(random.randint(60,600))


Itération 0
Vote pour  Alfred   Blondlot


f:\python37\lib\site-packages\ipykernel_launcher.py:105: DeprecationWarning: use driver.switch_to.frame instead


NoSuchElementException: Message: Unable to locate element: [id="mail"]
